<a href="https://colab.research.google.com/github/YujingYang666777/TaxonomyConstruction/blob/master/Project_CTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 80.6 MB/s 
     |████████████████████████████████| 7.6 MB 60.0 MB/s 


In [ ]:
import random
from typing import Union

In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import transformers
from transformers import AutoModel, AutoTokenizer

# Utilities

In [ ]:
# create sentence like: [s] is a subclass of [o]
def create_sentece_parent(parent, child):
  prompts = "Determine the hierarchy relationship between 2 concepts. Concept A is: {sub}. Concept B is {obj}. {sub} is a part of {obj} and {sub} is a subclass of {obj}."

  return prompts.format(sub = str(child), obj = str(parent))

In [ ]:
create_sentece_parent('CS', 'ML')

'Determine the hierarchy relationship between 2 concepts. Concept A is: ML. Concept B is CS. ML is a part of CS and ML is a subclass of CS.'

# CSKG: Preprocessing

In [ ]:
cskg = pd.read_csv("/content/drive/MyDrive/Master/CSKG/Query_sup_at_least5_no_cycle2.csv")
cskg = cskg[["sub", "obj", "sup"]]

In [ ]:
cskg.head()

,sub,obj,sup
0,http://scholkg.kmi.open.ac.uk/cskg/resource/co...,http://scholkg.kmi.open.ac.uk/cskg/resource/po...,5
1,http://scholkg.kmi.open.ac.uk/cskg/resource/el...,http://scholkg.kmi.open.ac.uk/cskg/resource/bo...,5
2,http://scholkg.kmi.open.ac.uk/cskg/resource/st...,http://scholkg.kmi.open.ac.uk/cskg/resource/co...,5
3,http://scholkg.kmi.open.ac.uk/cskg/resource/fi...,http://scholkg.kmi.open.ac.uk/cskg/resource/in...,6
4,http://scholkg.kmi.open.ac.uk/cskg/resource/mo...,http://scholkg.kmi.open.ac.uk/cskg/resource/sm...,6


In [ ]:
dataset = {"claim":[], "label":[]}

In [ ]:
len(dataset['label'])

326244

In [ ]:
# dataset['claim'][108748]

In [ ]:
cskg.iloc[0,0].split("/")[-1].replace(">", "").replace("_", " ") # replace "_"

'computer vision'

In [ ]:
cskg.iloc[0,1].split("/")[-1].replace(">", "").replace("_", " ") # replace "_"

'pose estimation'

## sampling

In [ ]:
for i in range(cskg.shape[0]):
  superclass = cskg.iloc[i,0]
  subclass = cskg.iloc[i,1]
  parent = superclass.split("/")[-1].replace(">", "").replace("_", " ")
  child = subclass.split("/")[-1].replace(">", "").replace("_", " ")
  claim = create_sentece_parent(parent, child)
  dataset["claim"].append(claim)
  dataset["label"].append(1)
  if i % 1000 ==0:
    print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000


In [ ]:
for i in range(cskg.shape[0]):
  superclass = cskg.iloc[i,0]
  subclass = cskg.iloc[i,1]
  parent = superclass.split("/")[-1].replace(">", "").replace("_", " ")
  child = subclass.split("/")[-1].replace(">", "").replace("_", " ")
  claim = create_sentece_parent(child, parent)
  dataset["claim"].append(claim)
  dataset["label"].append(0)
  if i % 1000 ==0:
    print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000


## Negative sampling

In [ ]:
import networkx as nx
G = nx.DiGraph()

for index, row in cskg.iterrows():
  #t = (row['sub'], row['obj'])
  G.add_edge(row['sub'], row['obj'], weight=row['sup'])

In [ ]:
length = cskg.shape[0]

In [ ]:
i = 0
while True:
  index_1 = random.randint(0, length-1)
  index_2 = random.randint(0, length-1)
  col_1 = random.randint(0, 1)
  col_2 = random.randint(0, 1)
  # print(index_1, col_1)
  # print(index_2, col_2)
  parent = cskg.iloc[index_1,col_1]
  child = cskg.iloc[index_2,col_2]
  has_path = nx.has_path(G,parent,child)
  if not has_path:
    claim = create_sentece_parent(parent, child)
    dataset["claim"].append(claim)
    dataset["label"].append(0)
    i += 1
    if i % 1000 ==0:
      print(i)
    if i >= length:
      break
  # break


# CSO pre-processing

In [ ]:
cso = pd.read_csv("/content/drive/MyDrive/Master/CSO.3.3.csv", dtype = str)

In [ ]:
columns = cso.columns

In [ ]:
relations = (cso[columns[1]].unique())



In [ ]:
relations

array(['<http://cso.kmi.open.ac.uk/schema/cso#superTopicOf>',
       '<http://cso.kmi.open.ac.uk/schema/cso#contributesTo>',
       '<http://cso.kmi.open.ac.uk/schema/cso#relatedEquivalent>',
       '<http://www.w3.org/2000/01/rdf-schema#label>',
       '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
       '<http://cso.kmi.open.ac.uk/schema/cso#preferentialEquivalent>',
       '<http://www.w3.org/2002/07/owl#sameAs>',
       '<http://schema.org/relatedLink>'], dtype=object)

In [ ]:
filter = cso.loc[cso[columns[1]] == relations[1]] # only is-a relation

In [ ]:
filter

,<https://cso.kmi.open.ac.uk/topics/computer_science>,<http://cso.kmi.open.ac.uk/schema/cso#superTopicOf>,<https://cso.kmi.open.ac.uk/topics/artificial_intelligence>
4,<https://cso.kmi.open.ac.uk/topics/nonrigid_re...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/rigid_regis...
9,<https://cso.kmi.open.ac.uk/topics/non-rigid_r...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/rigid_regis...
10,<https://cso.kmi.open.ac.uk/topics/automated_p...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/numerical_a...
11,<https://cso.kmi.open.ac.uk/topics/automated_p...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/manifold_le...
15,<https://cso.kmi.open.ac.uk/topics/nonlinear_d...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/isomap>
...,...,...,...
91750,<https://cso.kmi.open.ac.uk/topics/partial_dif...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/inverse_pro...
91751,<https://cso.kmi.open.ac.uk/topics/partial_dif...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/difference_...
91752,<https://cso.kmi.open.ac.uk/topics/partial_dif...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/distributed...
91753,<https://cso.kmi.open.ac.uk/topics/partial_dif...,<http://cso.kmi.open.ac.uk/schema/cso#contribu...,<https://cso.kmi.open.ac.uk/topics/convection_...


## adding test dataset

In [ ]:
test_dataset = {"claim":[], "label":[]}

In [ ]:
len(test_dataset['label'])

0

In [ ]:
test_dataset['label'][-10:]

[]

In [ ]:
test_dataset['claim'][:10]

[]

In [ ]:
# positive sampling
length = filter.shape[0]
count = 0
while True:
  # i = random.randint(0, length - 1)
  i = count
  superclass = filter[columns[0]].tolist()[i]
  subclass = filter[columns[2]].tolist()[i]
  sup = superclass.split("/")[-1].replace(">", "").replace("_", " ")
  sub = subclass.split("/")[-1].replace(">", "").replace("_", " ")
  claim = create_sentece_parent(sup, sub)
  test_dataset["claim"].append(claim)
  test_dataset["label"].append(1)
  
  count += 1
  if count % 1000 ==0:
    print(count)

  if count >= 48000:
    break

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000


In [ ]:
# negative sampling
length = filter.shape[0]
count = 0
while True:
  # i = random.randint(0, length - 1)
  i = random.randint(0, length-1)
  superclass = filter[columns[0]].tolist()[i]
  subclass = filter[columns[2]].tolist()[i]
  parent = superclass.split("/")[-1].replace(">", "").replace("_", " ")
  child = subclass.split("/")[-1].replace(">", "").replace("_", " ")
  claim = create_sentece_parent(child, parent)
  test_dataset["claim"].append(claim)
  test_dataset["label"].append(0)
  
  count += 1
  if count % 1000 ==0:
    print(count)

  if count >= length / 2:
    break

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


### negative smapling

In [ ]:
import networkx as nx
G = nx.DiGraph()

for index, row in filter.iterrows():
  #t = (row['sub'], row['obj'])
  G.add_edge(row[columns[0]], row[columns[2]], weight=1)

In [ ]:
totol_concept = filter[columns[0]].unique().tolist() + filter[columns[2]].unique().tolist()

In [ ]:
i = 0
length = len(totol_concept)
while True:
  index_1 = random.randint(0, length-1)
  index_2 = random.randint(0, length-1)

  sub = totol_concept[index_1]
  obj = totol_concept[index_2]
  has_path = nx.has_path(G,sub,obj)
  if not has_path:
    claim = create_sentece_parent(sub, obj)
    test_dataset["claim"].append(claim)
    test_dataset["label"].append(0)
    i += 1
    if i % 1000 ==0:
      print(i)
    if i >= 24000:
      break

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


## Get is-a relations

In [ ]:
relations[0]

In [ ]:
filter = cso.loc[cso[columns[1]] == relations[1]] # only is-a relation

In [ ]:
filter.head(10)


In [ ]:
subject = filter[columns[0]].tolist()[0]

In [ ]:
subject.split("/")[-1].replace(">", "").replace("_", " ") # replace "_"

In [ ]:
for i in range(len(filter)):
  superclass = filter[columns[0]].tolist()[i]
  subclass = filter[columns[2]].tolist()[i]
  sup = superclass.split("/")[-1].replace(">", "").replace("_", " ")
  sub = subclass.split("/")[-1].replace(">", "").replace("_", " ")
  claim = createSentece(sub, sup)
  dataset["claim"].append(claim)
  dataset["label"].append(1)
  if i % 1000 ==0:
    print(i)

## Get non-is_a relation

In [ ]:
filter = cso.loc[cso[columns[1]] == relations[2]] # only is-a relation

In [ ]:
for i in range(len(filter)):
  superclass = filter[columns[0]].tolist()[i]
  subclass = filter[columns[2]].tolist()[i]
  sup = superclass.split("/")[-1].replace(">", "").replace("_", " ")
  sub = subclass.split("/")[-1].replace(">", "").replace("_", " ")
  claim = createSentece(sup, sub)
  # print(claim)
  dataset["claim"].append(claim)
  dataset["label"].append(0)
  if i % 1000 ==0:
    print(i)

# Save dict

In [ ]:

# load pickle module
import pickle

# define dictionary


# create a binary pickle file 
f = open("/content/drive/MyDrive/599/project_data/train_dataset.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(dataset,f)

# close file
f.close()

# create a binary pickle file 
f = open("/content/drive/MyDrive/599/project_data/test_dataset.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(test_dataset,f)

# close file
f.close()

In [ ]:
import pickle

with open('/content/drive/MyDrive/599/project_data/train_dataset.pkl', 'rb') as handle:
    train_dataset = pickle.load(handle)
    print(len(train_dataset["label"]))


with open('/content/drive/MyDrive/599/project_data/test_dataset.pkl', 'rb') as handle:
    test_dataset = pickle.load(handle)
    print(len(test_dataset["label"]))

326244
96201


# Pre-trained Language Model_baseline



In [ ]:
def freeze_params(model):
    # TODO: your work below
    pass
    for para in model.parameters():
      para.requires_grad = False

In [ ]:
class CustomDistilBert(nn.Module):
    def __init__(self):
        super().__init__()

        pass
        self.distilbert = transformers.AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
        self.tokenizer = transformers.AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        dim = self.distilbert.config.hidden_size
        
        # freeze bert
        # 

        self.pred_layer = nn.Linear(dim,1)  
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.BCELoss()
    # vvvvv DO NOT CHANGE BELOW THIS LINE vvvvv
    def get_distilbert(self):
        return self.distilbert
    def get_tokenizer(self):
        return self.tokenizer
    def get_pred_layer(self):
        return self.pred_layer
    def get_sigmoid(self):
        return self.sigmoid
    
    def get_criterion(self):
        return self.criterion
    # ^^^^^ DO NOT CHANGE ABOVE THIS LINE ^^^^^
    def assign_optimizer(self, **kwargs):
        
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer

    def slice_cls_hidden_state(
        self, x: transformers.modeling_outputs.BaseModelOutput
    ) -> torch.Tensor:

        return x.last_hidden_state[:,0,:]


    def tokenize(
        self,
        premise: "list[str]",
        # hypothesis: "list[str]",
        max_length: int = 128,
        truncation: bool = True,
        padding: bool = True,
    ):
        
        model_tokenize = self.get_tokenizer()
        return model_tokenize(premise, max_length=max_length, truncation=truncation, padding=padding,
                         return_tensors='pt')

    def forward(self, inputs: transformers.BatchEncoding):
        
        model = self.get_distilbert()
        pred = self.get_pred_layer()
        sigmoid = self.get_sigmoid()

        output = model(**inputs) #  output of your tokenizer
        output = self.slice_cls_hidden_state(output)
        output = pred(output)
        output = sigmoid(output)
        output = torch.squeeze(output)
        
        return output

In [ ]:
@torch.no_grad()
def eval_distilbert(model, loader, device):
    model.eval()
    targets = []
    preds = []
    for claim, target in tqdm(loader):
        preds.append(model(model.tokenize(claim).to(device)))
        targets.append(target)
    return torch.cat(preds), torch.cat(targets)

def train_distilbert(model, loader, device):
    model.train()
    criterion = model.get_criterion()
    total_loss = 0.0

    targets = []
    preds = []

    for claim, label in tqdm(loader):
        optimizer.zero_grad()
        inputs = model.tokenize(claim).to(device)
        target = label.to(device, dtype=torch.float32)
        pred = model(inputs)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # print(model.pred_layer.grad)

        preds.append(pred)
        targets.append(target)

    return total_loss / len(loader), torch.cat(preds), torch.cat(targets)




class NLIDataset(torch.utils.data.Dataset):
    def __init__(self, data_dict: dict):
        self.data_dict = data_dict
        dd = data_dict
        if len(dd["claim"]) != len(dd["label"]):
            raise AttributeError("Incorrect length in data_dict")
    def __len__(self):
        return len(self.data_dict["claim"])
    def __getitem__(self, idx):
        dd = self.data_dict
        return dd["claim"][idx], dd["label"][idx]

In [ ]:
def load_datasets(data_directory: str) -> Union[dict, dict]:
    """
    Reads the training and validation splits from disk and load
    them into memory.
    Parameters
    ----------
    data_directory: str
        The directory where the data is stored.
    
    Returns
    -------
    train: dict
        The train dictionary with keys 'premise', 'hypothesis', 'label'.
    validation: dict
        The validation dictionary with keys 'premise', 'hypothesis', 'label'.
    """
    import json
    import os
    with open(os.path.join(data_directory, "train.json"), "r") as f:
        train = json.load(f)
    with open(os.path.join(data_directory, "validation.json"), "r") as f:
        valid = json.load(f)
    return train, valid

## Training

In [ ]:
claim = ["one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", 'asd' ]
label = [0,1,0,0,1,1,0,1,0,1,1,1, 0]
dataset = {'claim':claim, 'label':label}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomDistilBert().to(device)
freeze_params(model.get_distilbert()) 

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score  # Make sure sklearn is installed
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# random.seed(2022)
# torch.manual_seed(2022)

# Parameters
sample_size = None  # Change this if you want to take a subset of data for testing
batch_size = 128
n_epochs = 5
num_words = 50000

# If you use GPUs, use the code below:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ###################### PART 1: TEST CODE ######################
# Prefilled code showing you how to use the helper functions
# train_raw, valid_raw = load_datasets("data/nli")



train_raw = {}
if sample_size is not None:
    for key in ["claim", "label"]:
        train_raw[key] = dataset[key][:sample_size]
        
    test_raw = train_raw
else:
  train_raw = train_dataset
  test_raw = test_dataset

print("=" * 80)
print("Running test code for part 1")
print("-" * 80)


train_loader = torch.utils.data.DataLoader(
    NLIDataset(train_raw), batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    NLIDataset(test_raw), batch_size=batch_size, shuffle=False
)





Running test code for part 1
--------------------------------------------------------------------------------


In [ ]:
len(train_raw['label'])

326244

In [ ]:
model = model.to(device)
optimizer = model.assign_optimizer(lr=1e-4)
train_acc_list = []
train_f1_list = []
test_acc_list = []
test_f1_list = []
for epoch in range(n_epochs):
    loss, preds, targets = train_distilbert(model, train_loader, device=device)
    preds = preds.round()
    train_acc = accuracy_score(targets.detach().cpu(), preds.detach().cpu())
    train_acc_list.append(train_acc)
    score = f1_score(targets.detach().cpu(), preds.detach().cpu())
    train_f1_list.append(score)


    preds, targets = eval_distilbert(model, test_loader, device=device)
    preds = preds.round()
    acc = accuracy_score(targets.cpu(), preds.cpu())
    test_acc_list.append(acc)
    f1 = f1_score(targets.cpu(), preds.cpu())
    test_f1_list.append(f1)

    print("Epoch:", epoch)
    print("Training loss:", loss)
    print("Training acc:", train_acc)
    print("Train f1:", score)


    print("Test acc:", acc)
    print("Test f1:", f1)
    torch.save(model.state_dict(), "/content/drive/MyDrive/599/project_data/baseline_model")
    print("model saved")

  0%|          | 0/2549 [00:00<?, ?it/s]

  0%|          | 0/752 [00:00<?, ?it/s]

Epoch: 0
Training loss: 0.4490354719022621
Training acc: 0.7251688919949486
Train f1: 0.5823652708556524
Test acc: 0.7314788827558965
Test f1: 0.755526953361599
model saved


  0%|          | 0/2549 [00:00<?, ?it/s]

  0%|          | 0/752 [00:00<?, ?it/s]

...

  0%|          | 0/752 [00:00<?, ?it/s]

Epoch: 3
Training loss: 0.446268692817254
Training acc: 0.7287827515601819
Train f1: 0.5881005693217949
Test acc: 0.73694660138668
Test f1: 0.7576285796379657
model saved


  0%|          | 0/2549 [00:00<?, ?it/s]

  0%|          | 0/752 [00:00<?, ?it/s]

Epoch: 4
Training loss: 0.445148378049406
Training acc: 0.7310540576991454
Train f1: 0.591965996391302
Test acc: 0.7383083335932059
Test f1: 0.7617650677088756
model saved


## Plot

In [ ]:
print(train_acc_list)

In [ ]:
print(train_f1_list)

In [ ]:
print(test_acc_list)

In [ ]:
print(test_f1_list)

In [ ]:

import matplotlib.pyplot as plt
epochs = range(1,11)
plt.plot(epochs, test_acc_list, 'g', label='Test Accuracy_score')
plt.plot(epochs, train_acc_list, 'b', label='Train Accuracy_score')
plt.title('Accuracy_score vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy_score')
plt.legend()
plt.show()

## Test acc

In [ ]:
from sklearn.metrics import accuracy_score

preds, targets = eval_distilbert(model, test_loader, device=device)
preds = preds.round()
acc = accuracy_score(targets.cpu(), preds.cpu())
print(acc)

  0%|          | 0/752 [00:00<?, ?it/s]

0.7282876477375495


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

preds, targets = eval_distilbert(model, test_loader, device=device)
preds = preds.round()
acc = accuracy_score(targets.cpu(), preds.cpu())
print(acc)

cm = confusion_matrix(targets.cpu(),preds.cpu())

In [ ]:
cm

In [ ]:
import array

In [ ]:
cm = [[50000,     0], [16692, 31308]]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
# X, y = make_classification(random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                     random_state=0)
# clf = SVC(random_state=0)
# clf.fit(X_train, y_train)

# predictions = clf.predict(X_test)
# cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
# plt.figure(figsize=(30,30))  

cm = cm

plt = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[1,0])
# plt.plot()

display(plt.plot())

## Inference

In [ ]:
model.state_dict()
model.to(device)

In [ ]:
# e.g.
# computer science, machine learning
# rnn, ir, natural language processing
# computer vision

subclass = "sddsssssddd"
superclass = "x"

claim = create_sentece_parent(superclass, subclass)
print(claim)
preds = (model(model.tokenize(claim).to(device)))
print(preds)
print("=" * 80)
claim = create_sentece_parent(subclass, superclass)
print(claim)
preds = (model(model.tokenize(claim).to(device)))
print(preds)

## save/load model

In [ ]:
def save_mode(model, path_name):
  prefix = "/content/drive/MyDrive/599/project_data/baseline_model_"
  full_path = prefix + path_name
  torch.save(model.state_dict(), full_path)

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/599/project_data/baseline_model")

In [ ]:
model = CustomDistilBert()
model.load_state_dict(torch.load("/content/drive/MyDrive/599/project_data/baseline_model"))
model.eval()

In [ ]:
freeze_params(model.get_distilbert()) 

In [ ]:
model.state_dict()
model = model.to(device)

# soft prompt model

In [ ]:
def pad_attention_mask(mask, p):
    # TODO: your work below
    pass
    mask = torch.nn.functional.pad(mask,(p,0,0,0), value = 1)
    return mask

In [ ]:
class SoftPrompting(nn.Module):
    def __init__(self, p: int, e: int):
        super().__init__()
        self.p = p
        self.e = e
        self.prompts = nn.Parameter(torch.randn((p, e)),requires_grad=True)
        # self.prompts = torch.randn((p, e),requires_grad=True)
        
    def forward(self, embedded):
        # TODO: your work below
        pass
        p = self.p
        prompts = self.prompts
        # [batch, word, dim]
        result = torch.nn.functional.pad(embedded,(0,0,p,0))
        for i in range(p):
          result[:,i,:] = prompts[i,:]
          
        return result

In [ ]:
class CustomSciBert(nn.Module):
    def __init__(self):
        super().__init__()

        # TODO: your work below
        pass
        self.distilbert = transformers.AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
        self.tokenizer = transformers.AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        dim = self.distilbert.config.hidden_size

          
        #dim = self.distilbert.config.dim
        self.pred_layer = nn.Linear(dim,1)  
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.BCELoss()
    # vvvvv DO NOT CHANGE BELOW THIS LINE vvvvv
    def get_distilbert(self):
        return self.distilbert
    def get_tokenizer(self):
        return self.tokenizer
    def get_pred_layer(self):
        return self.pred_layer
    def get_sigmoid(self):
        return self.sigmoid
    
    def get_criterion(self):
        return self.criterion
    # ^^^^^ DO NOT CHANGE ABOVE THIS LINE ^^^^^
    def assign_optimizer(self, **kwargs):
        # TODO: your work below
        pass
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer

    def slice_cls_hidden_state(
        self, x: transformers.modeling_outputs.BaseModelOutput
    ) -> torch.Tensor:
        # TODO: your work below
        pass
        return x.last_hidden_state[:,0,:]


    def tokenize(
        self,
        premise: "list[str]",
        # hypothesis: "list[str]",
        max_length: int = 128,
        truncation: bool = True,
        padding: bool = True,
    ):
        # TODO: your work below
        pass
        model_tokenize = self.get_tokenizer()
        return model_tokenize(premise, max_length=max_length, truncation=truncation, padding=padding,
                         return_tensors='pt')

    def forward(self, inputs: transformers.BatchEncoding):
        # TODO: your work below
        pass
        model = self.get_distilbert()
        pred = self.get_pred_layer()
        sigmoid = self.get_sigmoid()

        output = model(inputs_embeds = inputs.input_embedded, attention_mask = inputs.attention_mask)
        output = self.slice_cls_hidden_state(output)
        output = pred(output)
        output = sigmoid(output)
        output = torch.squeeze(output)
        
        return output

## Traning

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_SoftPrompting(sp, model, loader, device):
    model.train()
    sp.train()
    criterion = model.get_criterion()
    # optimizer = model.assign_optimizer()
    total_loss = 0.0
    optimizer = torch.optim.Adam([{'params': model.parameters(), 'lr': 1e-4},
                                  {'params': sp.parameters(), 'lr': 1e-4}])
    # print(sp.prompts)
    # print("-"*80)
    # print(model.pred_layer.weight)
    for claim, label in tqdm(loader):
        optimizer.zero_grad()

        batch = model.tokenize(claim).to(device)
        batch.input_embedded = sp(model.get_distilbert().embeddings(batch.input_ids))
        batch.attention_mask = pad_attention_mask(batch.attention_mask, 5)
        # inputs = model.tokenize(premise, hypothesis).to(device)
        
        target = label.to(device, dtype=torch.float32)
        pred = model(batch)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    # print(sp.prompts.grad)
    return total_loss / len(loader), pred, target

In [ ]:
@torch.no_grad()
def eval_SoftPrompting(sp, model, loader, device):
    model.eval()
    sp.eval()
    targets = []
    preds = []
    for claim, target in tqdm(loader):

        batch = model.tokenize(claim).to(device)
        batch.input_embedded = sp(model.get_distilbert().embeddings(batch.input_ids))
        batch.attention_mask = pad_attention_mask(batch.attention_mask, 5)
        # inputs = model.tokenize(premise, hypothesis).to(device)
        
        pred = model(batch).to(device)

        preds.append(pred)
        targets.append(target)
    return torch.cat(preds), torch.cat(targets)

In [ ]:
claim = ["one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one", "one" ]
label = [1,1,1,1,1,1, 1,1,1,1,1,1]
dataset = {'claim':claim, 'label':label}

In [ ]:

import pandas as pd
from sklearn.metrics import f1_score  # Make sure sklearn is installed
from sklearn.metrics import accuracy_score

sample_size = None  # Change this if you want to take a subset of data for testing. Using None to train whole dataset
batch_size = 128
n_epochs = 10
num_words = 50000

# If you use GPUs, use the code below:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ###################### PART 1: TEST CODE ######################
# Prefilled code showing you how to use the helper functions


train_raw = {}
if sample_size is not None:
    for key in ["claim", "label"]:
        train_raw[key] = dataset[key][:sample_size]
    test_raw =   dataset 
else:
  train_raw = train_dataset
  test_raw =  test_dataset 

# ###################### PART 2: TEST CODE ######################

model = CustomSciBert().to(device)
freeze_params(model.get_distilbert()) # Now, model should have no trainable parameters

sp = SoftPrompting(p=5, e=model.get_distilbert().embeddings.word_embeddings.embedding_dim).to(device)


train_loader = torch.utils.data.DataLoader(
    NLIDataset(train_raw), batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    NLIDataset(test_dataset), batch_size=batch_size, shuffle=False
)


train_f1_scores = []
train_acc_list = []
train_loss = []

test_f1_scores = []
test_acc_list = []
# upload a plot of the validation loss, 
# and another of the validation accuracy over each of the 10 epochs you trained the model for


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
len(train_raw['label'])

326244

In [ ]:
for epoch in range(n_epochs):

    loss, preds, targets = train_SoftPrompting(sp, model, train_loader, device=device)

    train_loss.append(loss)

    preds = preds.round()
    trainACC = accuracy_score(targets.detach().cpu(), preds.detach().cpu())
    train_acc_list.append(trainACC)
    
    trainF1 = f1_score(targets.detach().cpu(), preds.detach().cpu())
    train_f1_scores.append(trainF1)

    # test
    preds, targets = eval_SoftPrompting(sp, model, test_loader, device=device)
    preds = preds.round()
    targets = targets.cpu().detach().numpy()
    preds = preds.cpu().detach().numpy()

    f1score = f1_score(targets, preds)
    test_f1_scores.append(f1score)

    testACC = accuracy_score(targets, preds)
    test_acc_list.append(testACC)

    print("Epoch:", epoch)
    print("Training loss:", loss)
    print("Train acc:", trainACC)
    print("Train F1:", trainF1)

    print("Test acc:", testACC)
    print("Test F1:", f1score)
    print( )

  0%|          | 0/2549 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## model save

In [ ]:
torch.save(sp.state_dict(), "/content/drive/MyDrive/599/project_data/sp")
torch.save(model.state_dict(), "/content/drive/MyDrive/599/project_data/model_sp")

NameError: ignored

## plot

In [ ]:
# first 10 epochs's data

In [ ]:
train_acc_list = [0.67, 0.69, 0.75, 0.7,0.66,0.68,0.81, 0.71, 0.77,0.77]

In [ ]:
train_f1_list = [0.45901639344262296, 0.5753424657534247,0.6153846153846154, 0.5945945945945945,0.5405405405405406,
                 0.5428571428571428, 0.6885245901639343, 0.6233766233766234, 0.676056338028169, 0.6567164179104478]

In [ ]:
test_acc_list = [0.6885919199215726,0.7300758474794902,0.7274340849285382,0.7545740673855839, 0.7494246942882204,
                 0.762850214127238,0.7742118569733244, 0.7811361642846086,0.780465404261906,0.7824570455600847]

In [ ]:
test_f1_list = [0.69164035437295, 0.7654816873627114, 0.7514281143244336, 0.7787977715152024,0.7505137267795495
                0.7630751466540204, 0.7788201043225101, 0.7878781817272591, 0.7839808289839769, 0.7858209637599439]

# Xnetwork

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

In [ ]:
G.add_nodes_from([2, 3])

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
G.add_edges_from([(1, 2), (1, 3)])

In [ ]:
G = nx.Graph()

# SPARQL

In [ ]:
!pip install sparqlwrapper

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "http://vocabs.ardc.edu.au/repository/api/sparql/"
    "csiro_international-chronostratigraphic-chart_geologic-time-scale-2020"
)
sparql.setReturnFormat(JSON)

# gets the first 3 geological ages
# from a Geological Timescale database,
# via a SPARQL endpoint
sparql.setQuery("""
    PREFIX gts: <http://resource.geosciml.org/ontology/timescale/gts#>

    SELECT *
    WHERE {
        ?a a gts:Age .
    }
    ORDER BY ?a
    LIMIT 3
    """
)

try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "https://scholkg.kmi.open.ac.uk/sparqlendpoint/"
)
sparql.setReturnFormat(JSON)

# gets the first 3 geological ages
# from a Geological Timescale database,
# via a SPARQL endpoint
sparql.setQuery("""
    ## Example query: Select all statements about Wikipedia.  
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX cskg: <http://scholkg.kmi.open.ac.uk/cskg/resource/> # CS-KG resources 
    PREFIX cskg-ont: <http://scholkg.kmi.open.ac.uk/cskg/ontology#> # CS-KG ontology 
    PREFIX provo: <http://www.w3.org/ns/prov#> 
    PREFIX cso: <http://cso.kmi.open.ac.uk/schema/cso#> 
    
    # ?type_sub ?pre ?obj ?type_obj ?sup
    SELECT ?sub ?type_sub ?pre ?obj ?type_obj ?sup FROM <http://scholkg.kmi.open.ac.uk/cskg> 
    WHERE { #?t rdf:subject cskg:computer_science . 
      ?t rdf:subject ?sub . 
      ?t rdf:predicate ?pre . 
      ?t rdf:object ?obj . 
      ?sub rdf:type ?type_sub .  
      ?obj rdf:type ?type_obj . 
      ?t cskg-ont:hasSupport ?sup . 
      FILTER(?sup =  and ?sup < 5)} 
    """
)

try:
    ret = sparql.queryAndConvert()
    print(len(ret["results"]["bindings"]))

except Exception as e:
    print(e)

In [ ]:
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX cskg: <http://scholkg.kmi.open.ac.uk/cskg/resource/> # CS-KG resources 
PREFIX cskg-ont: <http://scholkg.kmi.open.ac.uk/cskg/ontology#> # CS-KG ontology 
PREFIX provo: <http://www.w3.org/ns/prov#> 
PREFIX cso: <http://cso.kmi.open.ac.uk/schema/cso#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT (COUNT(?sub) as ?count)
WHERE {{ 
  ?t rdf:subject ?sub . ?t rdf:predicate ?pred. ?t rdf:object ?obj . 
  ?sub rdf:type cskg-ont:{subj} .  ?obj rdf:type cskg-ont:{obj}. ?t cskg-ont:hasSupport ?sup . 
  ?pred rdfs:subPropertyOf ?super .
  ?inverse owl:inverseOf ?super .
  FILTER(?sup > 5)
  FILTER(?super {equal} ?inverse)
}} LIMIT 10
"""

"""
SELECT ?sup (COUNT(?sup) as ?count)
WHERE { 
  ?t rdf:subject ?sub . ?t rdf:predicate ?pred. ?t rdf:object ?obj . ?t cskg-ont:hasSupport ?sup . 
  FILTER(?sup < 5)
} GROUP BY ?sup ORDER BY ?sup
"""

In [ ]:
!pip install matplotlib -U

In [ ]:
print(query.format(subj='Method', obj='Method', equal='='))

In [ ]:
types = ['Material', 'Method', 'Metric', 'OtherEntity', 'Task']
signs = ['=', '!=']

In [ ]:
def run_query(query):
    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()
        return ret["results"]["bindings"]
    except Exception as e:
        print(e)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm.notebook import tqdm
from collections import defaultdict

sparql = SPARQLWrapper(
    "https://scholkg.kmi.open.ac.uk/sparqlendpoint/"
)
sparql.setReturnFormat(JSON)

result = defaultdict(dict)


for sub in tqdm(types):
    for obj in tqdm(types):
        double_count = run_query(query.format(subj=sub, obj=obj, equal='='))
        count = int(double_count[0]['count']['value']) // 2
        single_count = run_query(query.format(subj=sub, obj=obj, equal='!='))
        count += int(single_count[0]['count']['value'])
        result[sub][obj] = count

In [ ]:
result = {'Method': {'Material': 168149,
              'Method': 856197,
              'Metric': 77112,
              'OtherEntity': 377030,
              'Task': 197729},
             'Metric': {'Material': 5103,
              'Method': 57115,
              'Metric': 13565,
              'OtherEntity': 12426,
              'Task': 11595},
             'OtherEntity': {'Material': 41231,
              'Method': 245739,
              'Metric': 18795,
              'OtherEntity': 166719,
              'Task': 13631},
             'Task': {'Material': 65558,
              'Method': 291844,
              'Metric': 22515,
              'OtherEntity': 83641,
              'Task': 85642},
             'Material': {'Material': 18166,
              'Method': 19450,
              'Metric': 4761,
              'OtherEntity': 34994,
              'Task': 9510}}

In [ ]:
m = [[0 for _ in range(5)] for _ in range(5)]
for i, sub in enumerate(types):
    for j, obj in enumerate(types):
        m[i][j] = result[sub][obj]

m = np.array(m)

In [ ]:
m = m / 1000

In [ ]:
m

In [ ]:
row_sum = m.sum(0, keepdims=True)
column_sum = m.sum(1, keepdims=True)


In [ ]:
m = m / (column_sum @ row_sum)

In [ ]:
m

In [ ]:
import numpy as np
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

m = np.array(m).round(5)
fig, ax = plt.subplots(figsize=(10,10))
im = ax.imshow(m)

ax.set_xticks(np.arange(len(types)), labels=types)
ax.set_yticks(np.arange(len(types)), labels=types)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(types)):
    for j in range(len(types)):
        text = ax.text(j, i, m[i, j],
                       ha="center", va="center", color="w")

## HISTOGRAM

In [ ]:
import pandas as pd


In [ ]:
queries = pd.read_csv("Query.csv")

In [ ]:
queries

# Load CSKG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import random

In [ ]:
CSKG_task = pd.read_csv("/content/drive/MyDrive/Master/CSKG/Query_sup_at_least5.csv")

In [ ]:
CSKG_task


In [ ]:
CSKG_cut_task = CSKG_task[["sub", "obj", "sup"]]

In [ ]:
CSKG_cut_task.shape

In [ ]:
import networkx as nx
G = nx.DiGraph()

In [ ]:
for index, row in CSKG_cut_task.iterrows():
  #t = (row['sub'], row['obj'])
  G.add_edge(row['sub'], row['obj'], weight=row['sup'])
  

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
CSKG_cut_task['sub'][0]
CSKG_cut_task['obj'][0]


In [ ]:
index = random.randint(0, 363143)
index

In [ ]:
print(nx.has_path(G,CSKG_cut_task['sub'][index],CSKG_cut_task['obj'][index]))
print(nx.has_path(G,CSKG_cut_task['obj'][index],CSKG_cut_task['sub'][index]))
print(CSKG_cut_task['sub'][index])
print(CSKG_cut_task['obj'][index])

In [ ]:
cycles = nx.simple_cycles(G)
cycles_2_list = []

In [ ]:
count = 0
len_2 = 0
for c in cycles:
  length = (len(c))
  # break
  # print(length)
  if  length == 2:
    print(c[0], c[-1])
    len_2 += 1
    cycles_2_list.append(c)

  count += 1
  if count > 5000: 
    print("-"*80)
    print(len_2)
    break


In [ ]:
def count_appearance(ent, dataFrame):
  tmp = dataFrame.loc[(dataFrame['sub'] == ent) ]
  return tmp.shape[0]

In [ ]:
count = 0
for c in cycles_2_list:
  sub1 = c[0]
  sub2 = c[1]
  sub1_2_weight = G[sub1][sub2]
  sub2_1_weight = G[sub2][sub1]

  if sub1_2_weight['weight'] == sub2_1_weight['weight']:
    df = dataFrame.loc[(dataFrame['sub'] == sub1) & (dataFrame['obj'] == sub2)]
    print(df.to_markdown())
    df = dataFrame.loc[(dataFrame['sub'] == sub2) & (dataFrame['obj'] == sub1)]
    print(df.to_markdown())
    
    sub1_count = count_appearance(sub1, dataFrame)
    sub2_count = count_appearance(sub2, dataFrame)
    print(sub1_count)
    print(sub2_count)
    if not (sub1_count == 0) and not( sub2_count == 0):
      count +=1
    if sub1_count == 1:
      toremove = dataFrame.loc[(dataFrame['sub'] == sub1) & (dataFrame['obj'] == sub2)].index.tolist()
      dataFrame = dataFrame.drop(toremove)

    if  sub2_count == 1:
      toremove = dataFrame.loc[(dataFrame['sub'] == sub2) & (dataFrame['obj'] == sub1)].index.tolist()
      dataFrame = dataFrame.drop(toremove)

    if sub1_count == sub2_count:
      toremove = dataFrame.loc[(dataFrame['sub'] == sub1) & (dataFrame['obj'] == sub2)].index.tolist()
      dataFrame = dataFrame.drop(toremove)

      toremove = dataFrame.loc[(dataFrame['sub'] == sub2) & (dataFrame['obj'] == sub1)].index.tolist()
      dataFrame = dataFrame.drop(toremove)

    if sub1_count < sub2_count:
      toremove = dataFrame.loc[(dataFrame['sub'] == sub1) & (dataFrame['obj'] == sub2)].index.tolist()
      dataFrame = dataFrame.drop(toremove)
    else:
      toremove = dataFrame.loc[(dataFrame['sub'] == sub2) & (dataFrame['obj'] == sub1)].index.tolist()
      dataFrame = dataFrame.drop(toremove)      


print(count)


In [ ]:
dataFrame = CSKG_cut_task.copy()

In [ ]:
dataFrame.shape

In [ ]:
true = [370, 82388, 18802, 28312, 17944 ]

In [ ]:
for c in cycles_2_list:
  sub1 = c[0]
  sub2 = c[1]
  sub1_2_weight = G[sub1][sub2]
  sub2_1_weight = G[sub2][sub1]
  df = dataFrame.loc[(dataFrame['sub'] == sub1) & (dataFrame['obj'] == sub2)]
  print(df.to_markdown())
  df = (dataFrame.loc[(dataFrame['sub'] == sub2) & (dataFrame['obj'] == sub1)])
  print(df.to_markdown())

  # remove the edge with smaller sup
  print("to remove!")
   # if sub1_2_weight['weight'] <= sub2_1_weight['weight']:
  toremove = dataFrame.loc[(dataFrame['sub'] == sub1) & (dataFrame['obj'] == sub2)].index.tolist()

  if len(list(set(toremove) & set(true))) == 0:

    dataFrame = dataFrame.drop(toremove)
  # elif sub2_1_weight['weight'] <= sub1_2_weight['weight']:
  toremove = dataFrame.loc[(dataFrame['sub'] == sub2) & (dataFrame['obj'] == sub1)].index.tolist()
  if len(list(set(toremove) & set(true))) == 0:
    
    dataFrame = dataFrame.drop(toremove)
  # else:
    #toremove = []
  #print(toremove)
  # dataFrame = dataFrame.drop(toremove)
  print()
  # break

In [ ]:
dataFrame.to_csv("/content/drive/MyDrive/Master/CSKG/Query_sup_at_least5_no_cycle2.csv")  

## DataFrame remove long cycles

In [ ]:
def read_cycle(list_nodes, G, dataFrame):
  edges = []
  length = len(list_nodes)
  for index in range(length):
    sub1 = list_nodes[index]
    sub2 = list_nodes[(index + 1) % length]
    edge = G[sub1][sub2]['weight']
    edges.append(edge)

  min_index = pd.Series(edges).idxmin()
  sub1 = list_nodes[min_index]
  sub2 = list_nodes[(min_index + 1) % length] 
  print(min_index)
  print(edges[min_index])
  print(sub1)
  print(sub2)
  toremove = dataFrame.loc[(dataFrame['sub'] == sub1) & (dataFrame['obj'] == sub2) & (dataFrame['sup'] == edges[min_index])].index.tolist()
  print(toremove)
  dataFrame = dataFrame.drop(toremove)
  return dataFrame, sub1, sub2

In [ ]:
dataFrame.to_csv("/content/drive/MyDrive/Master/CSKG/Hierarchy-SUP2-NO-CYCLES.csv")  

In [ ]:
dataFrame = pd.read_csv("/content/drive/MyDrive/Master/CSKG/Query-sup10.csv")  
dataFrame = dataFrame[["sub", "obj", "sup"]]

In [ ]:
dataFrame = pd.read_csv("/content/drive/MyDrive/Master/CSKG/Hierarchy-SUP2.csv")  
dataFrame = dataFrame[["sub", "obj", "sup"]]

In [ ]:
dataFrame = dataFrame.iloc[:3000]

In [ ]:

dataFrame

In [ ]:
import networkx as nx
G = nx.DiGraph()

for index, row in dataFrame.iterrows():
  #t = (row['sub'], row['obj'])
  G.add_edge(row['sub'], row['obj'], weight=row['sup'])

In [ ]:
import networkx as nx
G = nx.Graph()

for index, row in dataFrame.iterrows():
  #t = (row['sub'], row['obj'])
  G.add_edge(row['sub'], row['obj'], weight=1)

In [ ]:
cycles = nx.cycle_basis(G)
len(cycles)

In [ ]:
cycles = nx.recursive_simple_cycles(G)

In [ ]:
cycles = nx.simple_cycles(G)
cycles_list = []

In [ ]:
count = 0
len_2 = 0
for c in cycles:
  length = (len(c))
  # break
  print(length)
  # print(c[0], c[-1])
  cycles_list.append(length)

  count += 1
  if count >= 18643:
    print(count) 
    break

print('*'*80)
print(count)

In [ ]:
lengths = [len(c) for c in cycles]
bin = [2*i for i in range(100)]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# x = lengths
x = cycles_list

plt.figure(figsize=(20, 10))
plt.hist(x, bins =100 )
plt.title("# of cycles with different length for 310,000 graph")
plt.show() 


In [ ]:
G.number_of_edges()

In [ ]:
dataFrame.shape

In [ ]:
cycles = nx.simple_cycles(G)
cycles_list = []

In [ ]:
e = ('http://scholkg.kmi.open.ac.uk/cskg/resource/control_circuit', 'http://scholkg.kmi.open.ac.uk/cskg/resource/display_panel')
G.remove_edge(*e)

In [ ]:
i = 0

In [ ]:
while (i < 500):
  cycles = nx.simple_cycles(G)
  cycles_list = []

  for c in cycles:
    length = (len(c))
    # break
    print(length)
    print(c[0], c[-1])
    cycles_list.append(c)
    #if len(cycles_list) > 10:
    break

  #for cycle in cycles_list:
  # dataFrame, sub, obj = read_cycle(cycle[0], G, dataFrame)
  e = (cycles_list[0][-1], cycles_list[0][0])
  G.remove_edge(*e)
  print(dataFrame.shape)
  print(G.number_of_edges())
  i +=1

In [ ]:
i = 0

In [ ]:
while (i < 5):
  cycles = nx.simple_cycles(G)
  cycles_list = []

  for c in cycles:
    length = (len(c))
    # break
    print(length)
    print(c[0], c[-1])
    cycles_list.append(c)
    #if len(cycles_list) > 10:
    break

  #for cycle in cycles_list:
  dataFrame, sub, obj = read_cycle(cycle[0], G, dataFrame)
  e = (sub, obj)
  G.remove_edge(*e)
  print(dataFrame.shape)
  print(G.number_of_edges())
  i +=1 
  if i % 1000 ==0 :
    dataFrame.to_csv("/content/drive/MyDrive/Master/CSKG/Hierarchy-SUP2-NO-CYCLES_" + str(6000+i*100) + ".csv")  

In [ ]:
dataFrame.to_csv("/content/drive/MyDrive/Master/CSKG/Hierarchy-SUP2-NO-CYCLES_2.csv")  

In [ ]:
dataFrame, sub, obj = read_cycle(cycles_list[0], G, dataFrame)
e = (sub, obj)
G.remove_edge(*e)

# Randome

In [ ]:
import pandas as pd
# how to remove rows
# Create DataFrame
dataFrame = pd.DataFrame([[10, 15],[10, 15], [20, 25], [30, 35], [40, 45]],index=['w','q', 'x', 'y', 'z'],columns=['a', 'b'])

# DataFrame
print(dataFrame)

# deleting a row

print(dataFrame.loc[(dataFrame['a'] == 10) & (dataFrame['b'] == 15)].index.tolist())
toremove = dataFrame.loc[(dataFrame['a'] == 10) & (dataFrame['b'] == 15)].index.tolist()

dataFrame = dataFrame.drop(toremove)
print(dataFrame.to_markdown())

In [ ]:
import matplotlib.pyplot as plt
  
# values on x and y axis
x = [1, 2, 3, 4, 5]
y = [6, 7, 8, 9, 10]
  
# plot in it's default size
display(plt.plot(x, y))
  
# changing the size of figure to 2X2
plt.figure(figsize=(2, 2))
display(plt.plot(x, y))

In [ ]:
train = torch.load('/content/drive/MyDrive/599/project_data/train_loader_experiment.pth')

In [ ]:
from google.colab import runtime
runtime.unassign()